In [1]:
import pandas as pd
from datetime import datetime
import fsspec
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
# Load the datasets
yellow_taxi_data = pd.read_csv("D://Users//pshab//Downloads//Yellow_Taxi.csv")
green_taxi_data = pd.read_csv("D://Users//pshab//Downloads//Green_Taxi.csv")

In [3]:
yellow_taxi_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,"4/30/2023, 8:52:10 PM","4/30/2023, 9:05:26 PM",N,1,244,213,1,6.99,28.90,1.0,0.5,0.00,0.00,-,1.0,31.40,1,1,0.00
1,2,"4/30/2023, 8:29:49 PM","4/30/2023, 8:50:11 PM",N,1,33,100,1,6.60,30.30,1.0,0.5,5.00,0.00,-,1.0,40.55,1,1,2.75
2,2,"4/30/2023, 8:25:19 PM","4/30/2023, 8:32:12 PM",N,1,244,244,1,1.34,9.30,1.0,0.5,2.36,0.00,-,1.0,14.16,1,1,0.00
3,2,"4/30/2023, 8:07:06 PM","4/30/2023, 8:27:33 PM",N,5,82,75,1,7.79,22.73,0.0,0.0,2.29,6.55,-,1.0,32.57,1,1,0.00
4,2,"4/30/2023, 8:43:31 PM","4/30/2023, 8:46:59 PM",N,1,69,169,1,0.70,6.50,1.0,0.5,0.00,0.00,-,1.0,9.00,2,1,0.00


In [4]:
green_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,"4/30/2023, 8:33:13 PM","4/30/2023, 8:53:01 PM",0,7.80,1,N,138,43,1,33.8,7.75,0.5,8.60,0.00,1,51.65,0.0,1.75
1,1,"4/30/2023, 8:42:49 PM","4/30/2023, 9:11:18 PM",2,8.10,1,N,138,262,1,35.9,10.25,0.5,9.50,0.00,1,57.15,2.5,1.75
2,1,"4/30/2023, 8:56:34 PM","4/30/2023, 9:13:39 PM",2,9.10,1,N,138,141,1,35.2,10.25,0.5,10.70,6.55,1,64.20,2.5,1.75
3,2,"4/30/2023, 8:00:52 PM","4/30/2023, 8:20:12 PM",1,8.21,1,N,138,140,1,33.1,6.00,0.5,2.24,0.00,1,47.09,2.5,1.75
4,1,"4/30/2023, 8:05:50 PM","4/30/2023, 8:19:41 PM",0,7.90,1,N,138,263,1,31.0,10.25,0.5,9.85,6.55,1,59.15,2.5,1.75


In [5]:
# Convert date-time columns to datetime objects
yellow_taxi_data['lpep_pickup_datetime'] = pd.to_datetime(yellow_taxi_data['lpep_pickup_datetime'])
yellow_taxi_data['lpep_dropoff_datetime'] = pd.to_datetime(yellow_taxi_data['lpep_dropoff_datetime'])

green_taxi_data['tpep_pickup_datetime'] = pd.to_datetime(green_taxi_data['tpep_pickup_datetime'])
green_taxi_data['tpep_dropoff_datetime'] = pd.to_datetime(green_taxi_data['tpep_dropoff_datetime'])

C:\Users\pshab\AppData\Local\Temp\ipykernel_10148\4104758355.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  yellow_taxi_data['lpep_pickup_datetime'] = pd.to_datetime(yellow_taxi_data['lpep_pickup_datetime'])
C:\Users\pshab\AppData\Local\Temp\ipykernel_10148\4104758355.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  yellow_taxi_data['lpep_dropoff_datetime'] = pd.to_datetime(yellow_taxi_data['lpep_dropoff_datetime'])
C:\Users\pshab\AppData\Local\Temp\ipykernel_10148\4104758355.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  green_taxi_data['tpep_pickup_datetime']

In [6]:
# Handling missing values
yellow_taxi_data.dropna(inplace=True)
green_taxi_data.dropna(inplace=True)

# Remove rows with zero passenger counts
yellow_taxi_data = yellow_taxi_data[yellow_taxi_data['passenger_count'] > 0]
green_taxi_data = green_taxi_data[green_taxi_data['passenger_count'] > 0]

# Filter out trips with zero trip distance
yellow_taxi_data = yellow_taxi_data[yellow_taxi_data['trip_distance'] > 0]
green_taxi_data = green_taxi_data[green_taxi_data['trip_distance'] > 0]

# Filter out trips with zero fare amount
yellow_taxi_data = yellow_taxi_data[yellow_taxi_data['fare_amount'] > 0]
green_taxi_data = green_taxi_data[green_taxi_data['fare_amount'] > 0]

# Remove outliers in trip distance and fare amount (using z-score)
def remove_outliers(df, columns):
    z_scores = (df[columns] - df[columns].mean()) / df[columns].std()
    return df[(z_scores.abs() < 3).all(axis=1)]

columns_to_check = ['trip_distance', 'fare_amount']
yellow_taxi_data = remove_outliers(yellow_taxi_data, columns_to_check)
green_taxi_data = remove_outliers(green_taxi_data, columns_to_check)

# Convert categorical variables to appropriate data types
yellow_taxi_data['VendorID'] = yellow_taxi_data['VendorID'].astype('category')
yellow_taxi_data['store_and_fwd_flag'] = yellow_taxi_data['store_and_fwd_flag'].astype('category')
yellow_taxi_data['RatecodeID'] = yellow_taxi_data['RatecodeID'].astype('category')
yellow_taxi_data['payment_type'] = yellow_taxi_data['payment_type'].astype('category')
yellow_taxi_data['trip_type'] = yellow_taxi_data['trip_type'].astype('category')

green_taxi_data['VendorID'] = green_taxi_data['VendorID'].astype('category')
green_taxi_data['store_and_fwd_flag'] = green_taxi_data['store_and_fwd_flag'].astype('category')
green_taxi_data['RatecodeID'] = green_taxi_data['RatecodeID'].astype('category')
green_taxi_data['payment_type'] = green_taxi_data['payment_type'].astype('category')

In [7]:
cleaned_yellow_taxi_data=yellow_taxi_data
print(cleaned_yellow_taxi_data)

    VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0          2  2023-04-30 20:52:10   2023-04-30 21:05:26                  N   
1          2  2023-04-30 20:29:49   2023-04-30 20:50:11                  N   
2          2  2023-04-30 20:25:19   2023-04-30 20:32:12                  N   
3          2  2023-04-30 20:07:06   2023-04-30 20:27:33                  N   
4          2  2023-04-30 20:43:31   2023-04-30 20:46:59                  N   
..       ...                  ...                   ...                ...   
994        2  2023-05-01 09:44:37   2023-05-01 09:53:33                  N   
995        2  2023-05-01 09:22:00   2023-05-01 09:33:12                  N   
997        2  2023-05-01 09:01:12   2023-05-01 09:14:21                  N   
998        2  2023-05-01 09:43:54   2023-05-01 09:47:41                  N   
999        2  2023-05-01 09:19:10   2023-05-01 09:20:02                  N   

    RatecodeID  PULocationID  DOLocationID  passenger_count  tr

In [ ]:
cleaned_green_taxi_data=green_taxi_data
print(cleaned_green_taxi_data)